In [ ]:
import pandas as pd
from sqlalchemy import create_engine

db="sqlite:///top2020.db"
engine = create_engine(db,echo=False)

query_str = 'select e.id, e.user_id, u.name, e.album_id, a.name as album, a.genre_id, g.name as genre, e.position, e.score from entries as e join albums as a on e.album_id = a.id join genres as g on g.id = a.genre_id join users as u on e.user_id = u.id'

df = pd.read_sql(query_str,engine,index_col='id')
df.head(20)

In [ ]:
album_stats = pd.pivot_table(df, index = ['genre_id','genre','album_id','album'], values = ['score','position'], aggfunc=['count','sum','mean','min','max'])

album_ranking = album_stats.rank(method='dense',ascending=False)[('sum','score')]
album_genre_ranking = album_stats.groupby('genre').rank(method='dense',ascending=False)[('sum','score')]

album_results = pd.merge(album_stats, album_ranking, left_index=True, right_index=True)
album_results = pd.merge(album_results, album_genre_ranking, left_index=True, right_index=True)

map = {
    'sum_y': 'rank',
    'sum': 'genre_rank'
}
album_results.rename(map, axis=1, inplace=True)

drop_cols = [
    ('count','position'),
    ('sum_x','position')
]

album_results.drop(drop_cols, axis=1, inplace=True)
album_results.reset_index(inplace=True)
#album_results.swaplevel(0,1,axis=1)

album_results.columns = album_results.columns.map('|'.join).str.strip('|')

cols_map = {
    'rank|score':'rank',
    'album_id':'album_id',
    'album':'album',
    'genre_rank|score':'genre_rank',
    'genre_id':'genre_id',
    'genre':'genre',
    'count|score':'nb_votes',
    'sum_x|score':'total_score',
    'mean|score':'mean_score',
    'max|score':'highest_score',
    'min|score':'lowest_score',
    'mean|position':'mean_position',
    'min|position':'highest_position',
    'max|position':'lowest_position'
}
album_results = album_results.reindex(columns=cols_map.keys())
album_results.rename(cols_map, axis=1, inplace=True)
album_results.set_index('rank', inplace=True, drop=True)

#album_results.to_sql('album_results',engine)
#album_results.to_excel('top2020_album_results.xlsx')

album_results.head()

In [66]:
df_entries = pd.read_sql_table('entries',engine)
df_albums = pd.read_sql_table('albums',engine,index_col='id')
df_users = pd.read_sql_table('users',engine,index_col='id')
df_genres = pd.read_sql_table('genres',engine,index_col='id')

In [67]:
def build_entries():
    entries = pd.merge(df_entries, df_albums, left_on='album_id', right_on='id')
    entries = pd.merge(entries, df_genres, left_on='genre_id', right_on='id')

    return entries

entries = build_entries()
entries.head()

,id,name_x,user_id,album_id,position,score,name_y,genre_id,name
0,1,déhà - how to despise humanity in 7 lessons an...,1,1,1,30.000000,déhà - how to despise humanity in 7 lessons an...,24,Experimental
1,1082,déhà - how to despise humanity in 7 lessons an...,64,1,10,19.772727,déhà - how to despise humanity in 7 lessons an...,24,Experimental
2,26,melted bodies - enjoy yourself,6,26,4,27.413793,melted bodies - enjoy yourself,24,Experimental
3,541,melted bodies-enjoy yourself,33,26,2,29.137931,melted bodies - enjoy yourself,24,Experimental
4,1087,melted bodies - enjoy yourself,64,26,15,14.090909,melted bodies - enjoy yourself,24,Experimental


In [68]:
def format_entries(entries):

    map = {
        'id':'entry_id',
        'user_id':'user_id',
        'album_id':'album_id',
        'name_y':'album',
        'genre_id':'genre_id',
        'name':'genre',
        'position':'position',
        'score':'score'
    }

    drop_cols = (x for x in entries.columns if x not in map.keys())

    entries.drop(drop_cols, axis=1, inplace=True)
    entries.rename(map, axis=1, inplace=True)
    entries.set_index('entry_id', drop=False, inplace=True)

    return entries

entries = format_entries(entries)
entries.head()


,entry_id,user_id,album_id,position,score,album,genre_id,genre
entry_id,,,,,,,,
1,1,1,1,1,30.000000,déhà - how to despise humanity in 7 lessons an...,24,Experimental
1082,1082,64,1,10,19.772727,déhà - how to despise humanity in 7 lessons an...,24,Experimental
26,26,6,26,4,27.413793,melted bodies - enjoy yourself,24,Experimental
541,541,33,26,2,29.137931,melted bodies - enjoy yourself,24,Experimental
1087,1087,64,26,15,14.090909,melted bodies - enjoy yourself,24,Experimental


In [69]:
def build_album_results(entries):

    album_stats = pd.pivot_table(entries, index = ['genre_id','genre','album_id','album'], values = ['score','position'], aggfunc=['count','sum','mean','min','max'])

    album_ranking = album_stats.rank(method='dense',ascending=False)[('sum','score')]
    album_genre_ranking = album_stats.groupby('genre_id').rank(method='dense',ascending=False)[('sum','score')]

    album_results = pd.merge(album_stats, album_ranking, left_index=True, right_index=True)
    album_results = pd.merge(album_results, album_genre_ranking, left_index=True, right_index=True)

    album_results.reset_index(inplace=True)

    return album_results

album_results = build_album_results(entries)
album_results.head()

genre_id        genre album_id                                  album  \
                                                                          
0        1  Heavy Metal       19        cirith ungol - forever in black   
1        1  Heavy Metal      180             dynazty - the dark delight   
2        1  Heavy Metal      205              lonewolf - division hades   
3        1  Heavy Metal      213               avatar - hunter gatherer   
4        1  Heavy Metal      337  spirit adrift enlightened in eternity   

     count          sum_x                   mean                 min  \
  position score position       score   position      score position   
0        4     4       40   65.114943  10.000000  16.278736        3   
1        1     1       20   13.620690  20.000000  13.620690       20   
2        2     2       31   35.000000  15.500000  17.500000        1   
3        6     6       59  108.195489   9.833333  18.032581        1   
4        1     1       30    5.000000  30.000000   5.000000       30   

                  max             sum_y   sum  
       score position      score  score score  
0   5.000000       26  27.619048   64.0   2.0  
1  13.620690       20  13.620690  267.0   7.0  
2   5.000000       30  30.000000  148.0   3.0  
3  13.928571       13  30.000000   31.0   1.0  
4   5.000000       30   5.000000  306.0  11.0

In [70]:
def format_album_results(album_results):

    album_results.columns = album_results.columns.map('|'.join).str.strip('|')

    map = {
        'sum_y|score':'rank',
        'album_id':'album_id',
        'album':'album',
        'sum|score':'genre_rank',
        'genre_id':'genre_id',
        'genre':'genre',
        'count|score':'nb_votes',
        'sum_x|score':'total_score',
        'mean|score':'mean_score',
        'max|score':'highest_score',
        'min|score':'lowest_score',
        'mean|position':'mean_position',
        'min|position':'highest_position',
        'max|position':'lowest_position'
    }

    album_results = album_results.reindex(columns=map.keys())

    drop_cols = (x for x in album_results.columns if x not in map.keys())
    album_results.drop(drop_cols, axis=1, inplace=True)

    album_results.rename(map, axis=1, inplace=True)

    album_results.set_index('album_id', inplace=True, drop=True)

    return album_results

album_results = format_album_results(album_results)
album_results.to_excel('BLAH_albums.xlsx')

album_results.head()

,rank,album,genre_rank,genre_id,genre,nb_votes,total_score,mean_score,highest_score,lowest_score,mean_position,highest_position,lowest_position
album_id,,,,,,,,,,,,,
19,64.0,cirith ungol - forever in black,2.0,1,Heavy Metal,4,65.114943,16.278736,27.619048,5.000000,10.000000,3,26
180,267.0,dynazty - the dark delight,7.0,1,Heavy Metal,1,13.620690,13.620690,13.620690,13.620690,20.000000,20,20
205,148.0,lonewolf - division hades,3.0,1,Heavy Metal,2,35.000000,17.500000,30.000000,5.000000,15.500000,1,30
213,31.0,avatar - hunter gatherer,1.0,1,Heavy Metal,6,108.195489,18.032581,30.000000,13.928571,9.833333,1,13
337,306.0,spirit adrift enlightened in eternity,11.0,1,Heavy Metal,1,5.000000,5.000000,5.000000,5.000000,30.000000,30,30


In [77]:

def extend_entries(entries, album_results):

    album_keepcols = [
        'album_id',
        'total_score'
    ]
    album_dropcols = (x for x in album_results.columns if x not in album_keepcols)
    album_scores = album_results.drop(album_dropcols, axis=1)

    full_entries = pd.merge(entries, album_scores, on='album_id')

    full_entries = pd.merge(full_entries, df_users, left_on='user_id', right_on='id')

    full_entries.set_index('entry_id', inplace=True)
    cols = [
        'user_id',
        'name',
        'album_id',
        'album',
        'genre_id',
        'genre',
        'top_size',
        'position',
        'score',
        'total_score'
    ]

    full_entries = full_entries.reindex(columns=cols)

    return full_entries

full_entries = extend_entries(entries, album_results)
full_entries.head()

,user_id,name,album_id,album,genre_id,genre,top_size,position,score,total_score
entry_id,,,,,,,,,,
1,1,Denis Porcin,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,4,1,30.000000,49.772727
2,1,Denis Porcin,2,sepultura - quadra,2,Thrash Metal,4,2,21.666667,220.968801
3,1,Denis Porcin,3,jonathan hulten - chants from another place,29,Neofolk - DarkFolk,4,3,13.333333,43.333333
4,1,Denis Porcin,4,tomorrow's rain - hollow,11,Doom Metal,4,4,5.000000,32.578370
1082,64,Ugo Petropoulos,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,23,10,19.772727,49.772727


In [83]:
def compute_entry_stats(full_entries):

    full_entries['pop_score'] = full_entries['score'] * full_entries['total_score'] / 1000
    full_entries['edgyness'] = full_entries['top_size'] / full_entries['pop_score']

    return full_entries

full_entries = compute_entry_stats(full_entries)
full_entries.head()

,user_id,name,album_id,album,genre_id,genre,top_size,position,score,total_score,pop_score,edgyness
entry_id,,,,,,,,,,,,
1,1,Denis Porcin,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,4,1,30.000000,49.772727,1.493182,2.678843
2,1,Denis Porcin,2,sepultura - quadra,2,Thrash Metal,4,2,21.666667,220.968801,4.787657,0.835482
3,1,Denis Porcin,3,jonathan hulten - chants from another place,29,Neofolk - DarkFolk,4,3,13.333333,43.333333,0.577778,6.923077
4,1,Denis Porcin,4,tomorrow's rain - hollow,11,Doom Metal,4,4,5.000000,32.578370,0.162892,24.556170
1082,64,Ugo Petropoulos,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,23,10,19.772727,49.772727,0.984143,23.370598


In [79]:
def compute_user_stats(full_entries):
    
    user_genres = pd.pivot_table(full_entries, index=['name'], columns=['genre'], values=['score'], aggfunc=['sum'])
    user_edgyness = pd.pivot_table(full_entries, index = ['name'], values=['pop_score','edgyness'], aggfunc=['sum','mean'])

    return user_genres,user_edgyness

user_genres, user_edgyness = compute_user_stats(full_entries)
user_edgyness.head(20)

sum                   mean          
                        edgyness  pop_score    edgyness pop_score
name                                                             
Alex KM              2828.692511  36.233964   94.289750  1.207799
Alexandre Ardisson    484.207783   7.948679   53.800865  0.883187
Alexis Hérissant     5893.351206  22.179822  196.445040  0.739327
Amduscias Baal        411.161528  13.892519   41.116153  1.389252
Anthony Pwl            39.549452   7.167386    7.909890  1.433477
Arnaud Dehon         3071.751349  14.059001  139.625061  0.639046
Arno STROBL           238.537930  22.981257   19.878161  1.915105
Arsonist Tohliam       56.069242   7.660342    9.344874  1.276724
Arthur Bourson         61.916063   6.343350   20.638688  2.114450
Brett Caldas-Lima     124.951695   2.778727   41.650565  0.926242
Cam Iel               370.664347  18.978864   46.333043  2.372358
Caroline d'Avout      163.720186  10.296241   23.388598  1.470892
Corinne Royer béril  4185.996454  41.617442  139.533215  1.387248
Cretch                 16.303197   2.189848    5.434399  0.729949
Cyril Deparis        3579.333790  48.177191  119.311126  1.605906
Cyrille Druet         687.177193  33.201928   34.358860  1.660096
David Chardiet       4123.546964  72.554854  142.191275  2.501892
David Daz            2946.225191  32.708695   98.207506  1.090290
David Decaudin       3531.075492  14.856602  141.243020  0.594264
David Gibert          152.120228   8.640136   16.902248  0.960015

In [82]:
user_genres.head(20)

sum                                \
                                       score                                 
genre               Alternative Rock / Metal Avant-Garde Metal Black Metal   
name                                                                         
Alex KM                                  NaN         41.034483  104.137931   
Alexandre Ardisson                       NaN               NaN   95.625000   
Alexis Hérissant                   17.068966         41.896552  102.586207   
Amduscias Baal                           NaN               NaN   18.888889   
Anthony Pwl                              NaN               NaN         NaN   
Arnaud Dehon                             NaN               NaN  167.857143   
Arno STROBL                              NaN               NaN         NaN   
Arsonist Tohliam                         NaN               NaN   45.000000   
Arthur Bourson                           NaN               NaN   47.500000   
Brett Caldas-Lima                        NaN               NaN         NaN   
Cam Iel                                  NaN         12.142857   32.857143   
Caroline d'Avout                   25.833333               NaN         NaN   
Corinne Royer béril                      NaN               NaN   18.793103   
Cretch                                   NaN               NaN   17.500000   
Cyril Deparis                            NaN         24.655172  358.620690   
Cyrille Druet                            NaN         22.105263   37.631579   
David Chardiet                           NaN               NaN   22.857143   
David Daz                          91.379310               NaN   19.655172   
David Decaudin                           NaN               NaN         NaN   
David Gibert                             NaN               NaN         NaN   

                                                      \
                                                       
genre               Blues - Blues Rock Crust - Grind   
name                                                   
Alex KM                      25.689655           NaN   
Alexandre Ardisson                 NaN           NaN   
Alexis Hérissant                   NaN     43.620690   
Amduscias Baal                     NaN           NaN   
Anthony Pwl                        NaN           NaN   
Arnaud Dehon                       NaN           NaN   
Arno STROBL                        NaN     27.727273   
Arsonist Tohliam                   NaN           NaN   
Arthur Bourson                     NaN           NaN   
Brett Caldas-Lima                  NaN           NaN   
Cam Iel                            NaN     15.714286   
Caroline d'Avout                   NaN           NaN   
Corinne Royer béril          47.931034           NaN   
Cretch                             NaN           NaN   
Cyril Deparis                      NaN           NaN   
Cyrille Druet                      NaN           NaN   
David Chardiet               23.750000           NaN   
David Daz                          NaN           NaN   
David Decaudin               14.375000           NaN   
David Gibert                       NaN           NaN   

                                                                      \
                                                                       
genre               Dark Synth / Dungeon synth Death Metal Deathcore   
name                                                                   
Alex KM                                    NaN   30.000000       NaN   
Alexandre Ardisson                         NaN         NaN       NaN   
Alexis Hérissant                           NaN  156.896552       NaN   
Amduscias Baal                             NaN         NaN       NaN   
Anthony Pwl                                NaN         NaN       NaN   
Arnaud Dehon                               NaN   61.904762       NaN   
Arno STROBL                                NaN    5.000000       NaN   
Arsonist Tohliam               